In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import re
def clean_file(code):
  #no multiline comments
  clean_code=re.sub(r'/\*(.*?)\*/', '', code, flags=re.DOTALL)
  #no tabs
  clean_code=re.sub(r'\t',' ',clean_code)
  #no empty lines
  clean_code = re.sub(r'\n(\s*)\n', '\n', clean_code)
  clean_code= re.sub(r'^\n+','',clean_code)
  #lines of code
  matches=re.findall('\n',clean_code)
  return clean_code,len(matches)

In [6]:
import re

def count_loops(code):
    # Regular expression pattern to match loop structures
    loop_pattern = r'(?:for|while|do\s*while)'

    # Count the number of loop occurrences in the Java code
    loop_count = len(re.findall(loop_pattern, code))

    return loop_count

In [90]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def cosine_sim(code1,code2):
  data=[]
  data.append(re.sub(r'\n',' ',code1))
  data.append(re.sub(r'\n',' ',code2))
  vectorizer=TfidfVectorizer()
  X=vectorizer.fit_transform(data)
  tfidf_tokens=vectorizer.get_feature_names_out()
  df_tfidfvect = pd.DataFrame(data = X.toarray(),index = ['Doc1','Doc2'],columns = tfidf_tokens)
  return(cosine_similarity(X[0].toarray(),X[1].toarray())[0][0])

In [95]:
def buildingDF(original_path,code_path,value):
  original_file=open(original_path)
  og,og_loc=clean_file(original_file.read())

  code_file=open(code_path)
  code,code_loc=clean_file(code_file.read())

  dloc.append(abs(code_loc-og_loc))
  dnol.append(abs(count_loops(og)-count_loops(code)))

  csim.append(cosine_sim(og,code))
  classification.append(value)


Importing files

In [74]:
import glob
def file_access():
  files = glob.glob('/content/drive/MyDrive/ML Projects/SourceCode_PlagDetection/IR-Plag-Dataset/*', recursive = True)
  for file in files:

    original=glob.glob(file+'/original/*', recursive = True)
    #non_plag_code stores required file location

    non_plag=glob.glob(file+'/non-plagiarized/*', recursive = True)
    for non_plag_folders in non_plag:
        non_plag_code=glob.glob(non_plag_folders+'/*',recursive=True)
        #non_plag_code stores required file location
        buildingDF(original[0], non_plag_code[0],0) #0 implies non-plagiarized

    plagiarized=glob.glob(file+'/plagiarized/*', recursive = True)
    for plagiarized_folders in plagiarized:
      plagiarized_code_parent_folder=glob.glob(plagiarized_folders+'/*', recursive = True)
      for plagiarized_code_folder in plagiarized_code_parent_folder:
        plagiarized_code=glob.glob(plagiarized_code_folder+'/*', recursive = True)
        #plagiarized_code stores required file location
        buildingDF(original[0], plagiarized_code[0],1) #1 implies plagiarized


In [97]:
import pandas as pd

df=pd.DataFrame(columns=['Diff in LOC','Diff in No. of Loops','Cosine_Similarity','Class'])

dloc=[]
dnol=[]
csim=[]
classification=[]

file_access()


df['Diff in LOC']=dloc
df['Diff in No. of Loops']=dnol
df['Cosine_Similarity']=csim
df['Class']=classification


In [94]:
df

,Diff in LOC,Diff in No. of Loops,Cosine_Similarity,Class
0,5,0,0.766397,0
1,0,0,0.773519,0
2,3,0,0.869591,0
3,0,0,0.903035,0
4,4,0,0.860825,0
...,...,...,...,...
455,1,1,0.405223,1
456,1,1,0.546550,1
457,3,1,0.679137,1
458,3,0,0.428734,1


In [101]:
import numpy as np

X=df.drop('Class',axis=1)
y=df['Class']
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size =0.2,random_state=10)


In [102]:
print(X_train)

     Diff in LOC  Diff in No. of Loops  Cosine_Similarity
116            5                     0           0.909532
37            11                     0           0.757652
66             1                     0           0.740493
315           12                     0           0.783808
318            4                     0           0.801427
..           ...                   ...                ...
369            3                     1           0.463490
320           11                     0           0.610645
15             4                     0           0.685591
125            6                     0           0.873302
265            1                     0           0.471301

[368 rows x 3 columns]


In [118]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt = dt.fit(X_train,y_train)
y_pred2 = dt.predict(X_test)
print(accuracy_score(y_test,y_pred2))

0.7717391304347826
